In [1]:
import csv
import math
import pandas as pd
import numpy as np
from pyspark.sql.types import StringType
from pyspark.sql.functions import mean, col, udf
from pyspark import SQLContext, SparkContext
from sklearn.preprocessing import LabelEncoder, StandardScaler

String map setting

In [2]:
encoder = LabelEncoder()
scaler = StandardScaler()

Set interface of pyspark

In [2]:
Enter = SparkContext('local')
sqlContext = SQLContext(Enter)

f:\python\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Data input

In [3]:
traindata = sqlContext.read.csv('data/train.csv', sep='\t', header=True)
testdata = sqlContext.read.csv('data/test.csv', sep='\t', header=True)

Spark function

In [4]:
def preprocess(fea):
    global traindata, testdata
    ans = {}
    dummy = []

    for i in traindata.select(fea).distinct().collect():
        dummy.append(traindata[fea])
    for i in testdata.select(fea).distinct().collect():
        dummy.append(testdata[fea])

    testdata = testdata.drop('Step Start Time')
    traindata = traindata.drop('Step Start Time')

    testdata = testdata.drop('First Transaction Time')
    traindata = traindata.drop('First Transaction Time')

    testdata = testdata.drop('Correct Transaction Time')
    traindata = traindata.drop('Correct Transaction Time')

    testdata = testdata.drop('Step End Time')
    traindata = traindata.drop('Step End Time')

    testdata = testdata.drop('Step Duration (sec)')
    traindata = traindata.drop('Step Duration (sec)')

    testdata = testdata.drop('Correct Step Duration (sec)')
    traindata = traindata.drop('Correct Step Duration (sec)')

    testdata = testdata.drop('Error Step Duration (sec)')
    traindata = traindata.drop('Error Step Duration (sec)')

    testdata = testdata.drop('Incorrects')
    traindata = traindata.drop('Incorrects')

    testdata = testdata.drop('Hints')
    traindata = traindata.drop('Hints')

    testdata = testdata.drop('Corrects')
    traindata = traindata.drop('Corrects')

    cols = [i[fea]
            for i in traindata.union(testdata).select(fea).distinct().collect()]
    for index, sid in enumerate(cols):
        ans[sid] = index

    traindata = traindata.withColumn(fea, ans[traindata[fea]])
    testdata = testdata.withColumn(fea, ans[testdata[fea]])

def right_train():
    global traindata
    correct = traindata.filter(traindata['Correct First Attempt'] == '1')

def right_test():
    global testdata
    correct = testdata.filter(testdata['Correct First Attempt'] == '1')

def cal_rate_train(fea):
    global traindata, testdata
    col = traindata.groupby(fea).count()
    right = right_train().groupby(fea).count()

    rate = right.join(col, col[fea] == right[fea]).drop(col[fea]).select(fea,(right['count'] / col['count']))
    traindata = traindata.join(rate, rate[fea] == traindata[fea]).drop(rate[fea])
    testdata = testdata.join(rate, rate[fea] == testdata[fea]).drop(rate[fea])

def cal_rate_test(fea):
    global traindata, testdata
    col = testdata.groupby(fea).count()
    right = right_test().groupby(fea).count()
    
    rate = right.join(col, col[fea] == right[fea]).drop(col[fea]).select(fea,(right['count'] / col['count']))
    traindata = traindata.join(rate, rate[fea] == traindata[fea]).drop(rate[fea])
    testdata = testdata.join(rate, rate[fea] == testdata[fea]).drop(rate[fea])

Train data first round process

In [86]:
# for i in range(7,15):
#     preprocess(traindata[i])
#     preprocess(testdata[i])

with open("data/train.csv",newline = '') as f, open("data/first_pro.csv",'a+',newline='') as rf:
    reader = csv.reader(f, delimiter="\t")
    writ = csv.writer(rf)
    for row in reader: 
        writ.writerow(row)

Train data pre-process code

In [ ]:
df = pd.read_csv("data/first_pro.csv")
len_col = len(df['KC(Default)'])
index_tail = len_col
for i in range(0,len_col):
    if(type(df['KC(Default)'][i]) == type('a')):
        str_kc = df['KC(Default)'][i].split('~~')
        str_opp = df['Opportunity(Default)'][i].split('~~')
        lenko = len(str_kc)
        if(lenko > 1):
            for j in range(0,lenko):
                if j == 0:
                    df.loc[i,'KC(Default)'] = str_kc[0]
                    df.loc[i,'Opportunity(Default)'] = str_opp[0]
                    change_row = df.loc[i]
                else:
                    change_row[17] = str_kc[j]
                    change_row[18] = str_opp[j]
                    df.loc[index_tail] = change_row
                    index_tail += 1
df.to_csv("data/train_index.csv",index=False)

In [ ]:
df = pd.read_csv("data/first_pro.csv")
len_col = len(df['KC(Default)'])
index_tail = len_col
for i in range(0,len_col):
    if(type(df['KC(Default)'][i]) == type('a')):
        str_kc = df['KC(Default)'][i].split('~~')
        str_opp = df['Opportunity(Default)'][i].split('~~')
        lenko = len(str_kc)
        if(lenko > 1):
            for j in range(0,lenko):
                if j == 0:
                    df.loc[i,'KC(Default)'] = str_kc[0]
                    df.loc[i,'Opportunity(Default)'] = str_opp[0]
                    change_row = df.loc[i]
                else:
                    change_row[17] = str_kc[j]
                    change_row[18] = str_opp[j]
                    df.loc[index_tail] = change_row
                    index_tail += 1
df["KC(Default)"] = encoder.fit_transform(df["KC(Default)"])
df["Step Name"] = encoder.fit_transform(df["Step Name"])
df["Anon Student Id"] = encoder.fit_transform(df["Anon Student Id"])
df["Problem Hierarchy"] = encoder.fit_transform(df["Problem Hierarchy"])
df["Problem Name"] = encoder.fit_transform(df["Problem Name"])
df.to_csv("data/map_train_index.csv",index=False)

Test data first round process

In [4]:
# for i in range(7,15):
#     preprocess(traindata[i])
#     preprocess(testdata[i])

with open("data/test.csv",newline = '') as f, open("data/first_pro_test.csv",'a+',newline='') as rf:
    reader = csv.reader(f, delimiter="\t")
    writ = csv.writer(rf)
    for row in reader: 
        writ.writerow(row)

Test data pre-process code

In [ ]:
df = pd.read_csv("data/first_pro_test.csv")
len_col = len(df['KC(Default)'])
index_tail = len_col
for i in range(0,len_col):
    if(type(df['KC(Default)'][i]) == type('a')):
        str_kc = df['KC(Default)'][i].split('~~')
        str_opp = df['Opportunity(Default)'][i].split('~~')
        lenko = len(str_kc)
        if(lenko > 1):
            for j in range(0,lenko):
                if j == 0:
                    df.loc[i,'KC(Default)'] = str_kc[0]
                    df.loc[i,'Opportunity(Default)'] = str_opp[0]
                    change_row = df.loc[i]
                else:
                    change_row[17] = str_kc[j]
                    change_row[18] = str_opp[j]
                    df.loc[index_tail] = change_row
                    index_tail += 1
df.to_csv("data/test_index.csv",index=False)

In [ ]:
df = pd.read_csv("data/first_pro_test.csv")
len_col = len(df['KC(Default)'])
index_tail = len_col
for i in range(0,len_col):
    if(type(df['KC(Default)'][i]) == type('a')):
        str_kc = df['KC(Default)'][i].split('~~')
        str_opp = df['Opportunity(Default)'][i].split('~~')
        lenko = len(str_kc)
        if(lenko > 1):
            for j in range(0,lenko):
                if j == 0:
                    df.loc[i,'KC(Default)'] = str_kc[0]
                    df.loc[i,'Opportunity(Default)'] = str_opp[0]
                    change_row = df.loc[i]
                else:
                    change_row[17] = str_kc[j]
                    change_row[18] = str_opp[j]
                    df.loc[index_tail] = change_row
                    index_tail += 1
df["KC(Default)"] = encoder.fit_transform(df["KC(Default)"])
df["Step Name"] = encoder.fit_transform(df["Step Name"])
df["Anon Student Id"] = encoder.fit_transform(df["Anon Student Id"])
df["Problem Hierarchy"] = encoder.fit_transform(df["Problem Hierarchy"])
df["Problem Name"] = encoder.fit_transform(df["Problem Name"])
df.to_csv("data/map_test_index.csv",index=False)

Manipulate NaN

In [ ]:
train_filepath = 'data/train_index.csv'
rd = pd.read_csv(train_filepath)
traindata = pd.DataFrame(rd)

test_filepath = 'data/test_index.csv'
test_f = pd.read_csv(test_filepath)
testdata = pd.DataFrame(test_f)

for i in range(0, len(traindata['KC(Default)'])):
    if (type(traindata['KC(Default)'][i]) != type('a')):
        traindata['KC(Default)'][i] = -1
traindata.to_csv('data/train_select_str.csv')

for i in range(0, len(testdata['KC(Default)'])):
    if (type(testdata['KC(Default)'][i]) != type('a')):
        testdata['KC(Default)'][i] = -1

csv_path = 'data/test_select_str.csv'
df_new = testdata
x = np.array(testdata['Correct First Attempt'])
for i in range(0, len(x)):
    if math.isnan(x[i]):
        df_new = df_new.drop([i],axis=0)
df_new_cpy = df_new['Correct First Attempt']
df_new.to_csv(csv_path)

In [ ]:
train_filepath = 'data/map_train_index.csv'
rd = pd.read_csv(train_filepath)
traindata = pd.DataFrame(rd)

test_filepath = 'data/map_test_index.csv'
test_f = pd.read_csv(test_filepath)
testdata = pd.DataFrame(test_f)

for i in range(0, len(traindata['KC(Default)'])):
    if (type(traindata['KC(Default)'][i]) != type('a')):
        traindata['KC(Default)'][i] = -1
traindata.to_csv('data/train_select.csv')

for i in range(0, len(testdata['KC(Default)'])):
    if (type(testdata['KC(Default)'][i]) != type('a')):
        testdata['KC(Default)'][i] = -1

csv_path = 'data/test_select.csv'
df_new = testdata
x = np.array(testdata['Correct First Attempt'])
for i in range(0, len(x)):
    if math.isnan(x[i]):
        df_new = df_new.drop([i],axis=0)
df_new_cpy = df_new['Correct First Attempt']
df_new.to_csv(csv_path)

Single feature manipulate function

In [3]:
def rate_conv_s(data,pca,leng):
    a = np.array(data)
    b = np.unique(a)
    ans = {}
    rate = []
    for i in range(0,len(b)):
        dummy = [0,0,0]
        ans.setdefault(b[i],dummy)
    for i in range(0,leng):
        ans[data[i]][0] += 1
        if(pca[i] == 1):
            ans[data[i]][1] += 1
    for v in ans.values():
        v[2] = round(v[1] / v[0],5)
    for i in range(0,leng):
        rate.append(ans[data[i]][2])
    return rate

Multiple features manipulate function

In [4]:
def rate_conv_d(data1,data2,pca,leng):
    ans = {}
    rate = []
    for i in range(0,leng):
        zero = [0,0,0]
        dummy = (data1[i],data2[i])
        if(dummy not in ans):
            ans.setdefault(dummy,zero)
    for i in range(0,leng):
        ans[(data1[i],data2[i])][0] += 1
        if(pca[i] == 1):
            ans[(data1[i],data2[i])][1] += 1
    for v in ans.values():
        v[2] = round(v[1] / v[0],5)
    for i in range(0,leng):
        rate.append(ans[(data1[i],data2[i])][2])
    return rate

Train additional feature generation code

In [ ]:
df = pd.read_csv("data/train_select.csv")

sid = df["Anon Student Id"]
p_name = df["Problem Name"]
p_hier = df["Problem Hierarchy"]
s_name = df["Step Name"]
kc = df["KC(Default)"]
pca = df["Correct First Attempt"]

arr_leng = len(sid)
arr1 = rate_conv_s(sid,pca,arr_leng)
arr2 = rate_conv_s(p_name,pca,arr_leng)
arr3 = rate_conv_s(s_name,pca,arr_leng)
arr4 = rate_conv_s(kc,pca,arr_leng)
arr5 = rate_conv_s(p_hier,pca,arr_leng)
arr6 = rate_conv_d(sid,kc,pca,arr_leng)
arr7 = rate_conv_d(sid,p_name,pca,arr_leng)
arr8 = rate_conv_d(sid,s_name,pca,arr_leng)

df['rsid'] = arr1
df['rp_name'] = arr2
df['rs_name'] = arr3
df['rkc'] = arr4
df['rp_hier'] = arr5
df['rsid_kc'] = arr6
df['rsid_pname'] = arr7
df['rsid_sname'] = arr8

ans = df.loc[:,['rsid','rp_name','rs_name','rkc','rp_hier','rsid_kc','rsid_pname','rsid_sname']]
ans.to_csv("data/train_rate.csv",mode = 'a',index=False)

Test set rate fill(single feature)

In [33]:
def test_s(data1,data2,data3,leng,leng_t):
    a = np.array(data1)
    b = np.unique(a)

    ans = dict.fromkeys(b,-1)
    for i in range(0,leng):
        if ans[data1[i]] == -1:
            ans[data1[i]] = data2[i]
    fea = []
    for i in range(0,leng_t):
        if data3[i] in ans:
            fea.append(ans[data3[i]])
        else:
            fea.append(-1)
    return fea

Test set rate fill(multiple feature)

In [35]:
def test_d(data1,data2,data3,data4,data5,leng,leng_t):
    ans = {}
    for i in range(0,leng):
        dummy = (data1[i],data2[i])
        if dummy not in ans:
            ans.setdefault(dummy,-1)

    for i in range(0,leng):
        dummy = (data1[i],data2[i])
        if ans[dummy] == -1:
            ans[dummy] = data3[i]
    fea = []
    for i in range(0,leng_t):
        dummy = (data4[i],data5[i])
        if dummy in ans:
            fea.append(ans[dummy])
        else:
            fea.append(-1)
    return fea

Test set feature fill code

In [37]:
major = pd.read_csv("data/train_select_str.csv")
rate = pd.read_csv("data/train_rate.csv")
test = pd.read_csv("data/test_index.csv")

sid = major["Anon Student Id"]
p_name = major["Problem Name"]
p_hier = major["Problem Hierarchy"]
s_name = major["Step Name"]
kc = major["KC(Default)"]

rsid = rate["rsid"]
rp_name = rate["rp_name"]
rp_hier = rate["rp_hier"]
rs_name = rate["rs_name"]
rkc = rate["rkc"]
rsid_kc = rate["rsid_kc"]
rsid_pname = rate["rsid_pname"]
rsid_sname = rate["rsid_sname"]

tsid = test["Anon Student Id"]
tp_name = test["Problem Name"]
tp_hier = test["Problem Hierarchy"]
ts_name = test["Step Name"]
tkc = test["KC(Default)"]

leng = len(sid)
leng_t = len(tsid)

test['rsid'] = test_s(sid,rsid,tsid,leng,leng_t)
test['rp_name'] = test_s(p_name,rp_name,tp_name,leng,leng_t)
test['rs_name'] = test_s(s_name,rs_name,ts_name,leng,leng_t)
test['rkc'] = test_s(kc,rkc,tkc,leng,leng_t)
test['rp_hier'] = test_s(p_hier,rp_hier,tp_hier,leng,leng_t)
test['rsid_kc'] = test_d(sid,kc,rsid_kc,tsid,tkc,leng,leng_t)
test['rsid_pname'] = test_d(sid,p_name,rsid_pname,tsid,tp_name,leng,leng_t)
test['rsid_sname'] = test_d(sid,s_name,rsid_sname,tsid,ts_name,leng,leng_t)

test.to_csv("data/test_rate_nan.csv",mode = 'a',index=False)

Test set nan reset function

In [3]:
def set_avr(col,leng):
    num = 0
    ans = 0
    for i in range(0,leng):
        if col[i] != -1:
            num += 1
            ans += col[i]
    ans = round(ans/num,5)
    for i in range(0,leng):
        if col[i] == -1:
            col[i] = ans
    return col

Test set nan reset

In [5]:
test = pd.read_csv("data/test_rate_nan.csv")

leng = len(test["rsid"])

set_avr(test["rsid"],leng)
set_avr(test["rp_name"],leng)
set_avr(test["rp_hier"],leng)
set_avr(test["rs_name"],leng)
set_avr(test["rkc"],leng)
set_avr(test["rsid_kc"],leng)
set_avr(test["rsid_pname"],leng)
set_avr(test["rsid_sname"],leng)

test.to_csv("data/test_rate.csv",mode = 'a',index=False)

C:\Users\李宗泽\AppData\Local\Temp\ipykernel_1208\3350195655.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = ans


Three additional feature manipulation of trian data

In [32]:
df = pd.read_csv("data/first_pro.csv")

opp = df["Opportunity(Default)"]
len_col = len(opp)

dummy0 = [0] * len_col
dummy1 = [0] * len_col
dummy2 = [0] * len_col
dummy3 = [0] * len_col

for i in range(0,len_col):
    if(type(opp[i]) == type('a')):
        str_opp = opp[i].split('~~')
        str_opp = [int(x) for x in str_opp]
        l = len(str_opp)
        
        min_opp = min(str_opp)
        avr = 0
        for k in range(0,l):
            avr += str_opp[k]
        avr = round(avr / l,5)
        if(l == 1):
            dummy1[i] = l
            dummy2[i] = min_opp
            dummy3[i] = avr
        if(l > 1):
            for j in range(0,l):
                if j == 0:
                    dummy0[i] = str_opp[0]
                    dummy1[i] = l
                    dummy2[i] = min_opp
                    dummy3[i] = avr
                else:
                    dummy0.append(str_opp[j])
                    dummy1.append(l)
                    dummy2.append(min_opp)
                    dummy3.append(avr)

ans = pd.DataFrame({'Opportunity(Default)':dummy0,'kc_num':dummy1,'min_opp':dummy2,'mean_opp':dummy3})
ans.to_csv(r"data/train_additional_rate.csv",mode = 'a',index=False)

Three additional feature manipulation of test data

In [29]:
df = pd.read_csv("data/first_pro_test.csv")

opp = df["Opportunity(Default)"]
len_col = len(opp)

dummy0 = [0] * len_col
dummy1 = [0] * len_col
dummy2 = [0] * len_col
dummy3 = [0] * len_col

for i in range(0,len_col):
    if(type(opp[i]) == type('a')):
        str_opp = opp[i].split('~~')
        str_opp = [int(x) for x in str_opp]
        l = len(str_opp)
        
        min_opp = min(str_opp)
        avr = 0
        for k in range(0,l):
            avr += str_opp[k]
        avr = round(avr / l,5)
        if(l == 1):
            dummy1[i] = l
            dummy2[i] = min_opp
            dummy3[i] = avr
        if(l > 1):
            for j in range(0,l):
                if j == 0:
                    dummy0[i] = str_opp[0]
                    dummy1[i] = l
                    dummy2[i] = min_opp
                    dummy3[i] = avr
                else:
                    dummy0.append(str_opp[j])
                    dummy1.append(l)
                    dummy2.append(min_opp)
                    dummy3.append(avr)

ans = pd.DataFrame({'Opportunity(Default)':dummy0,'kc_num':dummy1,'min_opp':dummy2,'mean_opp':dummy3})
ans.to_csv(r"data/test_additional_rate.csv",mode = 'a',index=False)

1500
1500
1500
1500
